# homework: 教育培训行业销售聊天机器人

### 完成内容：
1、使用GPT-4生成咨询销售话术，并生成文件education_question_data.txt  
2、处理数据、文本分割、使用Faiss持久化话术问答对  
3、使用retriever获取问答信息，提取faiss中的话术回答。（设置k=3、设置threshold：0.7/0.8/0.85）  
4、当Faiss中没有合适回答时，使用GPT-4能力予以回复  

#### 1、使用 GPT-4 构造教育培训机构咨询话术，供销售参考

https://platform.openai.com/playground/p/9uHlnWvFAujGi7515Uur4dDl?model=gpt-4&mode=chat


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶尖的教育培训机构的销售顾问，主要培训课程包括雅思英语、python编程和初高中数理化等、现在培训职场新人。
请给出30条实用的销售话术。每条话术格式以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：（部分）

```
1. 
[客户问题]: 贵机构的雅思英语培训课程具体包括什么内容？
[销售回答]: 我们的雅思英语课程非常全面，不仅覆盖听、说、读、写四个方面，还结合实战演练，帮助您在真实环境中提高英语水平。它是专门为准备雅思考试的学生量身定制的学习方案。

2. 
[客户问题]: 为什么我要选择你们的初高中数理化课程？
[销售回答]: 不同于其他机构，我们拥有丰富经验的专业教师团队，他们不仅教书，更注重引导学生的学习方法和思维模式。我们的课程内容针对性强，有针对性地解决学生深度理解及考试应用的问题。

3. 
[客户问题]: 我编程一窍不通，Python编程是否适合我？
[销售回答]: 当然了，我们针对从未接触过编程的新手设计了基础课程，从编程语言基础到逻辑思维的培养，一步一步带你进入编程的世界。另外，Python是一种非常友好的编程语言，非常适合初学者。

4. 
[客户问题]: 你们的课程贵吗？
[销售回答]: 我们的课程价格在同类型课程中处于中等水平。但是考虑到我们的高质量教学，提供的学习材料，以及随课程附带的辅导服务，这其实是一笔非常值得投资的费用。

5. 
[客户问题]: 你们的教师资质如何呢？
[销售回答]: 我们所有的教师都是经过严格挑选的专家，在他们的领域内拥有丰富的教学经验和专业知识，可以为您提供质量上乘的教学体验。

6. 
[客户问题]: 我忙于工作，如果不能按时参加课程怎么办？
[销售回答]: 我们理解每个人的时间都很宝贵，所以我们提供了录播视频的服务，只要您有空，任何时候都可以学习。

7. 
[客户问题]: 如果我付费了，但上了课发现不适合我怎么办？
[销售回答]: 我们提供了灵活的退款政策，如果您在试听课程后觉得不适合自己，我们可以无条件退款。

8. 
[客户问题]: 这个Python课程有多难，我需要提前准备什么?
[销售回答]: 我们的Python课程分为初级和高级两个阶段，初级阶段无需任何基础，只要你有学习的热情，我们的教师会从零开始教你。

```


#### 2、使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [education_question_data.txt](education_question_data.txt) 文件中

In [1]:
with open("education_question_data.txt") as f:
    education_question_data = f.read()

### 使用 CharacterTextSplitter 来进行文本分割


In [2]:
from langchain.text_splitter import CharacterTextSplitter

In [3]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [4]:
docs = text_splitter.create_documents([education_question_data])

Created a chunk of size 120, which is longer than the specified 100
Created a chunk of size 123, which is longer than the specified 100
Created a chunk of size 129, which is longer than the specified 100
Created a chunk of size 101, which is longer than the specified 100
Created a chunk of size 102, which is longer than the specified 100
Created a chunk of size 103, which is longer than the specified 100
Created a chunk of size 127, which is longer than the specified 100
Created a chunk of size 116, which is longer than the specified 100
Created a chunk of size 118, which is longer than the specified 100
Created a chunk of size 105, which is longer than the specified 100
Created a chunk of size 122, which is longer than the specified 100
Created a chunk of size 106, which is longer than the specified 100
Created a chunk of size 104, which is longer than the specified 100
Created a chunk of size 117, which is longer than the specified 100
Created a chunk of size 135, which is longer tha

In [5]:
docs[0].page_content

'[客户问题]: 贵机构的雅思英语培训课程具体包括什么内容？\n[销售回答]: 我们的雅思英语课程非常全面，不仅覆盖听、说、读、写四个方面，还结合实战演练，帮助您在真实环境中提高英语水平。它是专门为准备雅思考试的学生量身定制的学习方案。'

In [6]:
len(docs)

30

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [7]:
#from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings # langchain0.1.13

# langchain_community.embeddings.openai.OpenAIEmbeddings
db = FAISS.from_documents(docs, OpenAIEmbeddings())

In [8]:
answer_list = db.similarity_search("你们的课程贵不贵")

In [9]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题]: 你们的课程贵吗？
[销售回答]: 我们的课程价格在同类型课程中处于中等水平。但是考虑到我们的高质量教学，提供的学习材料，以及随课程附带的辅导服务，这其实是一笔非常值得投资的费用。

[客户问题]
你们的价格是怎么计算的？
[销售回答]
我们的价格主要是依据课程的内容、时长以及教师资质进行计算的。我们承诺，每一分钱都会用在提升您的学习体验和教学质量上。

[客户问题]: 你们的课程时间安排是怎样的？
[销售回答]: 我们的课程时间非常灵活，我们有早班，中班和晚班，并且周末也有课程，可以根据您的时间自由选择。

[客户问题]
你们初高中数理化的课程难度如何？我担心孩子学不来。
[销售回答]
请您放心，我们的课程会根据学生的基础和接受能力进行适当调整，确保每位学生都能跟上课程的进度。我们重视激发学生的学习兴趣，用富有趣味性的教学方式，让学生在快乐中学习。



In [10]:
db.save_local("education_question_data")

#### 3、使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [11]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [12]:
docs = topK_retriever.get_relevant_documents("你们的课程贵不贵")
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]: 你们的课程贵吗？
[销售回答]: 我们的课程价格在同类型课程中处于中等水平。但是考虑到我们的高质量教学，提供的学习材料，以及随课程附带的辅导服务，这其实是一笔非常值得投资的费用。

[客户问题]
你们的价格是怎么计算的？
[销售回答]
我们的价格主要是依据课程的内容、时长以及教师资质进行计算的。我们承诺，每一分钱都会用在提升您的学习体验和教学质量上。

[客户问题]: 你们的课程时间安排是怎样的？
[销售回答]: 我们的课程时间非常灵活，我们有早班，中班和晚班，并且周末也有课程，可以根据您的时间自由选择。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [13]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.85}
)

In [14]:
docs = retriever.get_relevant_documents("你们的课程贵不贵")
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]: 你们的课程贵吗？
[销售回答]: 我们的课程价格在同类型课程中处于中等水平。但是考虑到我们的高质量教学，提供的学习材料，以及随课程附带的辅导服务，这其实是一笔非常值得投资的费用。



### 提取向量数据库中的`销售回答`

In [15]:
docs = retriever.get_relevant_documents("你们的课程贵不贵")
docs[0].page_content.split("\n")

['[客户问题]: 你们的课程贵吗？',
 '[销售回答]: 我们的课程价格在同类型课程中处于中等水平。但是考虑到我们的高质量教学，提供的学习材料，以及随课程附带的辅导服务，这其实是一笔非常值得投资的费用。']

#### 尝试各种问题

In [16]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [17]:
query = "雅思培训课程有吗"
print(sales(query, 0.75))

['[客户问题]: 贵机构的雅思英语培训课程具体包括什么内容？\n[销售回答]: 我们的雅思英语课程非常全面，不仅覆盖听、说、读、写四个方面，还结合实战演练，帮助您在真实环境中提高英语水平。它是专门为准备雅思考试的学生量身定制的学习方案。', '[客户问题]\n我想提高我的英语水平，但不确定你们的雅思英语课程是否适合我？\n[销售回答]\n没问题，我们的雅思英语课程会根据学员的具体情况进行量身定制，会涵盖听、说、读、写全方位的提升，向我们的专业导师表达您的需求，他们会为您设计最适合您的学习计划。', '[客户问题]\n你们的英语课程只针对雅思考试吗？\n[销售回答]\n我们的英语课程除了专门针对雅思考试，还有针对托福、商务英语、日常英语等不同需求的课程。无论你是要出国留学，还是需要提升工作用英语，我们都能为你定制合适的学习计划。', '[客户问题]: 你们的课程时间安排是怎样的？\n[销售回答]: 我们的课程时间非常灵活，我们有早班，中班和晚班，并且周末也有课程，可以根据您的时间自由选择。']


In [18]:
query = "你们python编程课程的价格如何"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

score:0.8 ans: ['[客户问题]\n你们的Python编程课程包括哪些内容？\n[销售回答]\n我们的Python编程课程覆盖了从基础到高级的所有内容，包括Python语法、数据结构、面向对象编程、常用库的使用等。同时我们注重实践操作，会让学员在完成具体的项目中，真正掌握和应用所学的知识。', '[客户问题]\n你们的价格是怎么计算的？\n[销售回答]\n我们的价格主要是依据课程的内容、时长以及教师资质进行计算的。我们承诺，每一分钱都会用在提升您的学习体验和教学质量上。']



score:0.75 ans: ['[客户问题]\n你们的Python编程课程包括哪些内容？\n[销售回答]\n我们的Python编程课程覆盖了从基础到高级的所有内容，包括Python语法、数据结构、面向对象编程、常用库的使用等。同时我们注重实践操作，会让学员在完成具体的项目中，真正掌握和应用所学的知识。', '[客户问题]\n你们的价格是怎么计算的？\n[销售回答]\n我们的价格主要是依据课程的内容、时长以及教师资质进行计算的。我们承诺，每一分钱都会用在提升您的学习体验和教学质量上。', '[客户问题]: 你们的课程贵吗？\n[销售回答]: 我们的课程价格在同类型课程中处于中等水平。但是考虑到我们的高质量教学，提供的学习材料，以及随课程附带的辅导服务，这其实是一笔非常值得投资的费用。', '[客户问题]: 这个Python课程有多难，我需要提前准备什么?\n[销售回答]: 我们的Python课程分为初级和高级两个阶段，初级阶段无需任何基础，只要你有学习的热情，我们的教师会从零开始教你。']

score:0.5 ans: ['[客户问题]\n你们的Python编程课程包括哪些内容？\n[销售回答]\n我们的Python编程课程覆盖了从基础到高级的所有内容，包括Python语法、数据结构、面向对象编程、常用库的使用等。同时我们注重实践操作，会让学员在完成具体的项目中，真正掌握和应用所学的知识。', '[客户问题]\n你们的价格是怎么计算的？\n[销售回答]\n我们的价格主要是依据课程的内容、时长以及教师资质进行计算的。我们承诺，每一分钱都会用在提升您的学习体验和教学质量上。', '[客户问题]: 你们的课程贵吗？\n[销售回答]: 我们的课程价格在同类型课程中处于中等水平。但是考虑到我们的高质量教学，提供的学习材料，以及随课程附带的辅导服务，这其实是一笔非常值得投资的费用。', '[客户问题]: 这个Python课程有多难，我需要提前准备什么?\n[销售回答]: 我们的Python课程分为初级和高级两个阶段，初级阶段无需任何基础，只要你有学习的热情，我们的教师会从零开始教你。']



#### 4、引入GPT-4使用大语言模型能力润色话术或者回复没有的问题

In [19]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5) # gpt-4-1106-preview
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [20]:
qa_chain.invoke({"query":"你们有雅思考试培训课程吗？课程安排如何？"}) # 使用GPT-4润色回答

{'query': '你们有雅思考试培训课程吗？课程安排如何？',
 'result': '是的，我们有专门针对雅思考试的培训课程。这个课程非常全面，覆盖听、说、读、写四个方面，并结合实战演练。关于课程安排，我们的时间非常灵活，有早班，中班和晚班，并且周末也有课程，可以根据您的时间自由选择。'}

In [21]:
print(sales("你们有雅思考试培训课程吗？课程安排如何？")[0]) # 没有引入GPT-4

[客户问题]
你们的英语课程只针对雅思考试吗？
[销售回答]
我们的英语课程除了专门针对雅思考试，还有针对托福、商务英语、日常英语等不同需求的课程。无论你是要出国留学，还是需要提升工作用英语，我们都能为你定制合适的学习计划。


## 加载 FAISS 向量数据库已有结果

In [22]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

edu_qa_db = FAISS.load_local("education_question_data", OpenAIEmbeddings(), allow_dangerous_deserialization=True)

In [23]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
edu_qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=edu_qa_db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

/root/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [24]:
edu_qa_chain({"query": "你们有雅思考试培训课程吗？课程安排如何？"})

/root/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'query': '你们有雅思考试培训课程吗？课程安排如何？',
 'result': '是的，我们提供雅思英语课程，课程非常全面，覆盖听、说、读、写四个方面，并结合实战演练。我们的课程时间安排非常灵活，我们有早班，中班和晚班，并且周末也有课程，可以根据您的时间自由选择。'}

In [25]:
# 输出内部 Chain 的日志
edu_qa_chain.combine_documents_chain.verbose = True
edu_qa_chain({"query": "你们有雅思考试培训课程吗？课程安排如何？"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '你们有雅思考试培训课程吗？课程安排如何？',
 'result': '是的，我们有专门针对雅思考试的培训课程。课程内容非常全面，覆盖听、说、读、写四个方面，并结合实战演练，帮助您在真实环境中提高英语水平。关于课程时间，我们非常灵活，我们有早班，中班和晚班，并且周末也有课程，可以根据您的时间自由选择。'}

In [26]:
# 返回向量数据库的检索结果
edu_qa_chain.return_source_documents = True
result = edu_qa_chain({"query": "你们有雅思考试培训课程吗？课程安排如何？"})
result



> Entering new StuffDocumentsChain chain...



> Finished chain.


{'query': '你们有雅思考试培训课程吗？课程安排如何？',
 'result': '是的，我们有专门针对雅思考试的培训课程，内容全面，覆盖听、说、读、写四个方面，并结合实战演练。关于课程时间，我们非常灵活，我们有早班，中班和晚班，并且周末也有课程，可以根据您的时间自由选择。',
 'source_documents': [Document(page_content='[客户问题]\n你们的英语课程只针对雅思考试吗？\n[销售回答]\n我们的英语课程除了专门针对雅思考试，还有针对托福、商务英语、日常英语等不同需求的课程。无论你是要出国留学，还是需要提升工作用英语，我们都能为你定制合适的学习计划。'),
  Document(page_content='[客户问题]\n我想提高我的英语水平，但不确定你们的雅思英语课程是否适合我？\n[销售回答]\n没问题，我们的雅思英语课程会根据学员的具体情况进行量身定制，会涵盖听、说、读、写全方位的提升，向我们的专业导师表达您的需求，他们会为您设计最适合您的学习计划。'),
  Document(page_content='[客户问题]: 贵机构的雅思英语培训课程具体包括什么内容？\n[销售回答]: 我们的雅思英语课程非常全面，不仅覆盖听、说、读、写四个方面，还结合实战演练，帮助您在真实环境中提高英语水平。它是专门为准备雅思考试的学生量身定制的学习方案。'),
  Document(page_content='[客户问题]: 你们的课程时间安排是怎样的？\n[销售回答]: 我们的课程时间非常灵活，我们有早班，中班和晚班，并且周末也有课程，可以根据您的时间自由选择。')]}